<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/vecstack_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings;warnings.simplefilter('ignore')
from sklearn.datasets import load_iris,load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from vecstack import stacking
import numpy as np
from sklearn.svm import SVC

# 載入資料
dataset = load_breast_cancer()
X, y = dataset.data[:,:], dataset.target
# 特徵縮放
X = StandardScaler().fit_transform(X)
# 切分資料
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 42)
print(X.shape,y.shape)

(569, 30) (569,)


# level1 model

In [0]:
# base model 1
model_1 = [LogisticRegression()]
oof_train_1, oof_test_1 = stacking(model_1,X_train[:,:], y_train, X_test[:,:],regression = False,metric = f1_score,n_folds = 5)
# base model 2
model_2 = [SVC()]
oof_train_2, oof_test_2 = stacking(model_2,X_train[:,:], y_train, X_test[:,:],regression = False,metric = f1_score,n_folds = 5)
# base model 3
model_3 = [KNeighborsClassifier()]
oof_train_3, oof_test_3 = stacking(model_3,X_train[:,:], y_train, X_test[:,:],regression = False,metric = f1_score,n_folds = 5)

# merge oof_train oof_test

In [3]:
oof_train = np.hstack([oof_train_1,oof_train_2,oof_train_3])
oof_test = np.hstack([oof_test_1,oof_test_2,oof_test_3])
print(oof_train.shape,oof_test.shape)

(455, 3) (114, 3)


# level2 model

In [4]:
meta_model = XGBClassifier().fit(oof_train, y_train)
y_pred = meta_model.predict(oof_test)
accuracy_score(y_test, y_pred)

0.9736842105263158

# 對照組

In [5]:
model = XGBClassifier().fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9649122807017544